# Creating a Sentiment Analysis Web App
## Using PyTorch and SageMaker

_Much of this notebook's markdown content was provided by Udacity's Deep Learning Nanodegree Program | Deployment_

---

Project goal: to have a simple web page which a user can use to enter a movie review. The web page will then send the review off to our deployed model which will predict the sentiment of the entered review.

## General Outline

1. Download the data (IMDB Movie reviews dataset).
2. Process / Prepare the data.
3. Upload the processed data to S3.
4. Train a chosen model.
5. Test the trained model (typically using a batch transform job).
6. Deploy the trained model.
7. Use the deployed model.

For this project, you will be following the steps in the general outline with some modifications. 

First, you will not be testing the model in its own step. You will still be testing the model, however, you will do it by deploying your model and then using the deployed model by sending the test data to it. One of the reasons for doing this is so that you can make sure that your deployed model is working correctly before moving forward.

In addition, you will deploy and use your trained model a second time. In the second iteration you will customize the way that your trained model is deployed by including some of your own code. In addition, your newly deployed model will be used in the sentiment analysis web app.

## Step 1: Downloading the data

We will be using the [IMDb dataset](http://ai.stanford.edu/~amaas/data/sentiment/)

> Maas, Andrew L., et al. [Learning Word Vectors for Sentiment Analysis](http://ai.stanford.edu/~amaas/data/sentiment/). In _Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies_. Association for Computational Linguistics, 2011.

In [1]:
%mkdir ../data
!wget -O ../data/aclImdb_v1.tar.gz http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -zxf ../data/aclImdb_v1.tar.gz -C ../data

!cd; cd SageMaker/sagemaker-deployment/data rm *.tar.gz

--2020-06-03 09:10:19--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘../data/aclImdb_v1.tar.gz’

../data/aclImdb_v1. 100%[===================>]  80.23M  19.7MB/s    in 6.7s    

2020-06-03 09:10:26 (12.0 MB/s) - ‘../data/aclImdb_v1.tar.gz’ saved [84125825/84125825]



## Step 2: Preparing and Processing the data

To begin with, we will read in each of the reviews and combine them into a single input structure. Then, we will split the dataset into a training set and a testing set.

In [2]:
import os
import glob
from time import strftime, gmtime

In [3]:
def read_imdb_data(data_dir='../data/aclImdb'):
    data = {}
    labels = {}
    
    for data_type in ['train', 'test']:
        data[data_type] = {}
        labels[data_type] = {}
        
        for sentiment in ['pos', 'neg']:
            data[data_type][sentiment] = []
            labels[data_type][sentiment] = []
            
            path = os.path.join(data_dir, data_type, sentiment, '*.txt')
            files = glob.glob(path)
            
            for f in files:
                with open(f) as review:
                    data[data_type][sentiment].append(review.read())
                    # Here we represent a positive review by '1' and a negative review by '0'
                    labels[data_type][sentiment].append(1 if sentiment == 'pos' else 0)
                    
            assert len(data[data_type][sentiment]) == len(labels[data_type][sentiment]), \
                    "{}/{} data size does not match labels size".format(data_type, sentiment)
                
    return data, labels

In [4]:
data, labels = read_imdb_data()
print("IMDB reviews: train = {} pos / {} neg, test = {} pos / {} neg".format(
            len(data['train']['pos']), len(data['train']['neg']),
            len(data['test']['pos']), len(data['test']['neg'])))

IMDB reviews: train = 12500 pos / 12500 neg, test = 12500 pos / 12500 neg


Now that we've read the raw training and testing data from the downloaded dataset, we will combine the positive and negative reviews and shuffle the resulting records.

In [5]:
from sklearn.utils import shuffle

def prepare_imdb_data(data, labels):
    """Prepare training and test sets from IMDb movie reviews."""
    
    #Combine positive and negative reviews and labels
    data_train = data['train']['pos'] + data['train']['neg']
    data_test = data['test']['pos'] + data['test']['neg']
    labels_train = labels['train']['pos'] + labels['train']['neg']
    labels_test = labels['test']['pos'] + labels['test']['neg']
    
    #Shuffle reviews and corresponding labels within training and test sets
    data_train, labels_train = shuffle(data_train, labels_train)
    data_test, labels_test = shuffle(data_test, labels_test)
    
    # Return a unified training data, test data, training labels, test labets
    return data_train, data_test, labels_train, labels_test

In [6]:
train_X, test_X, train_y, test_y = prepare_imdb_data(data, labels)
print("IMDb reviews (combined): train = {}, test = {}".format(len(train_X), len(test_X)))

IMDb reviews (combined): train = 25000, test = 25000


Now that we have our training and testing sets unified and prepared, we should do a quick check and see an example of the data our model will be trained on. This is generally a good idea as it allows you to see how each of the further processing steps affects the reviews and it also ensures that the data has been loaded correctly.

In [7]:
print(train_X[100])
print(train_y[100])

My family has watched Arthur Bach stumble and stammer since the movie first came out. We have most lines memorized. I watched it two weeks ago and still get tickled at the simple humor and view-at-life that Dudley Moore portrays. Liza Minelli did a wonderful job as the side kick - though I'm not her biggest fan. This movie makes me just enjoy watching movies. My favorite scene is when Arthur is visiting his fiancée's house. His conversation with the butler and Susan's father is side-spitting. The line from the butler, "Would you care to wait in the Library" followed by Arthur's reply, "Yes I would, the bathroom is out of the question", is my NEWMAIL notification on my computer. "Arthur is truly "funny stuff"!
1


In [8]:
print(test_X[100])
print(test_y[100])

Well, this movie wasn't as horrible as I thought it would be. I was expecting to give it one star. I chose to give it three. Why? Well, for a cheesy horror/sci-fi movie, it's not all that bad. Sure the characters are tacky (as is their acting - including a young Leonardo DiCaprio), the effects cheap looking, and the monsters...well let's just say that I've seen some more effective Halloween costumes in Elementary Schools... But there was something about this movie that made me watch it till the end. A little bit of humor helped out a bit I guess. And if you ever wanted to see an alien pass gas, this movie will let you live your dream. 3/10
0


The first step in processing the reviews is to make sure that any html tags that appear should be removed. BeautifulSoup is ideal for this purpose. In addition we wish to tokenize and stem our input, that way words such as *entertained* and *entertaining* are considered the same with regard to sentiment analysis. 

In [9]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *

import re
from bs4 import BeautifulSoup

In [10]:
def review_to_words(review):
    nltk.download("stopwords", quiet=True)
    stemmer = PorterStemmer()
    
    text = BeautifulSoup(review, "html.parser").get_text() # Remove HTML tags
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) # Convert to lower case
    words = text.split() # Split string into words
    words = [w for w in words if w not in stopwords.words("english")] # Remove stopwords
    words = [PorterStemmer().stem(w) for w in words] # stem
    
    return words

In [11]:
# example of our preprocessing function in action
review_to_words(train_X[100])

['famili',
 'watch',
 'arthur',
 'bach',
 'stumbl',
 'stammer',
 'sinc',
 'movi',
 'first',
 'came',
 'line',
 'memor',
 'watch',
 'two',
 'week',
 'ago',
 'still',
 'get',
 'tickl',
 'simpl',
 'humor',
 'view',
 'life',
 'dudley',
 'moor',
 'portray',
 'liza',
 'minelli',
 'wonder',
 'job',
 'side',
 'kick',
 'though',
 'biggest',
 'fan',
 'movi',
 'make',
 'enjoy',
 'watch',
 'movi',
 'favorit',
 'scene',
 'arthur',
 'visit',
 'fianc',
 'e',
 'hous',
 'convers',
 'butler',
 'susan',
 'father',
 'side',
 'spit',
 'line',
 'butler',
 'would',
 'care',
 'wait',
 'librari',
 'follow',
 'arthur',
 'repli',
 'ye',
 'would',
 'bathroom',
 'question',
 'newmail',
 'notif',
 'comput',
 'arthur',
 'truli',
 'funni',
 'stuff']

The function review_to_words : 
* removes stopwords found in nltk's english vocabulary stopwords set
* removes any NON-alphanumeric character using regular expressions, substituting a space instead
* transforms every word into lowercase
* splits the text into tokens, using space (' ') as a separator

* (sidenote: converting entertained and entertaining into entertain is *stemming* the words : it's worth noting that this whilst this method is very useful, since it looks for suffixes to remove, the word it returns is not always a grammatically correct one: e.g. 'ponies' will be stemmed to 'poni')

The method below applies the `review_to_words` method to each of the reviews in the training and testing datasets. In addition it caches the results. This is because performing this processing step can take a long time. This way if you are unable to complete the notebook in the current session, you can come back without needing to process the data a second time.

In [12]:
import pickle

cache_dir = os.path.join("../cache", "sentiment_analysis")  # where to store cache files
os.makedirs(cache_dir, exist_ok=True)  # ensure cache directory exists

def preprocess_data(data_train, data_test, labels_train, labels_test,
                    cache_dir=cache_dir, cache_file="preprocessed_data.pkl"):
    """Convert each review to words; read from cache if available."""

    # If cache_file is not None, try to read from it first
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = pickle.load(f)
            print("Read preprocessed data from cache file:", cache_file)
        except:
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        # Preprocess training and test data to obtain words for each review
        words_train = [review_to_words(review) for review in data_train]
        words_test = [review_to_words(review) for review in data_test]
        
        # Write to cache file for future runs
        if cache_file is not None:
            cache_data = dict(words_train=words_train, words_test=words_test,
                              labels_train=labels_train, labels_test=labels_test)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                pickle.dump(cache_data, f)
            print("Wrote preprocessed data to cache file:", cache_file)
    else:
        # Unpack data loaded from cache file
        words_train, words_test, labels_train, labels_test = (cache_data['words_train'],
                cache_data['words_test'], cache_data['labels_train'], cache_data['labels_test'])
    
    return words_train, words_test, labels_train, labels_test

In [13]:
# Preprocess data
train_X, test_X, train_y, test_y = preprocess_data(train_X, test_X, train_y, test_y)

Read preprocessed data from cache file: preprocessed_data.pkl


## Transform the data

We will represent each word as an integer. Of course, some of the words that appear in the reviews occur very infrequently and so likely don't contain much information for the purposes of sentiment analysis. The way we will deal with this problem is that we will fix the size of our working vocabulary and we will only include the words that appear most frequently. We will then combine all of the infrequent words into a single category and, in our case, we will label it as `1`.

Since we will be using a recurrent neural network, it's necessary for the length of each review to be the same. To do this, we will fix a size for our reviews and then pad short reviews with the category 'no word' (which we will label `0`) and truncate long reviews.

To begin with, we need to construct a way to map words that appear in the reviews to integers. Here we fix the size of our vocabulary (including the 'no word' and 'infrequent' categories) to be `5000` but you may wish to change this to see how it affects the model.

In [14]:
import numpy as np

In [15]:
def build_dict(data, vocab_size = 5000):
    """Construct and return a dictionary mapping each of the most frequently appearing words to a unique integer."""
    word_count = {}
    for review in data:
        for word in review:
            try:
                word_count[word] += 1
            except:
                word_count[word] = 1
     # A dict storing the words that appear in the reviews along with how often they occur 
    sorted_words = sorted(word_count.items(), key = lambda kv: kv[1], reverse=True)
    
    word_dict = {} # This is what we are building, a dictionary that translates words into integers
    for idx, word in enumerate(sorted_words[:vocab_size - 2]): # The -2 is so that we save room for the 'no word'
        word_dict[word[0]] = idx + 2                              # 'infrequent' labels
        
    return word_dict

In [16]:
word_dict = build_dict(train_X)

In [17]:
word_dict

{'movi': 2,
 'film': 3,
 'one': 4,
 'like': 5,
 'time': 6,
 'good': 7,
 'make': 8,
 'charact': 9,
 'get': 10,
 'see': 11,
 'watch': 12,
 'stori': 13,
 'even': 14,
 'would': 15,
 'realli': 16,
 'well': 17,
 'scene': 18,
 'look': 19,
 'show': 20,
 'much': 21,
 'end': 22,
 'peopl': 23,
 'bad': 24,
 'go': 25,
 'great': 26,
 'also': 27,
 'first': 28,
 'love': 29,
 'think': 30,
 'way': 31,
 'act': 32,
 'play': 33,
 'made': 34,
 'thing': 35,
 'could': 36,
 'know': 37,
 'say': 38,
 'seem': 39,
 'work': 40,
 'plot': 41,
 'two': 42,
 'actor': 43,
 'year': 44,
 'come': 45,
 'mani': 46,
 'seen': 47,
 'take': 48,
 'want': 49,
 'life': 50,
 'never': 51,
 'littl': 52,
 'best': 53,
 'tri': 54,
 'man': 55,
 'ever': 56,
 'give': 57,
 'better': 58,
 'still': 59,
 'perform': 60,
 'find': 61,
 'feel': 62,
 'part': 63,
 'back': 64,
 'use': 65,
 'someth': 66,
 'director': 67,
 'actual': 68,
 'interest': 69,
 'lot': 70,
 'real': 71,
 'old': 72,
 'cast': 73,
 'though': 74,
 'live': 75,
 'star': 76,
 'enjoy': 7

* The five most frequent words are 'movi', 'film', 'one', 'like', 'time'.

* The fact that words 1,2 and 4 appear frequently is unsurprising, given that the dataset is about movie reviews. 

In [18]:
[print(list(word_dict.keys())[list(word_dict.values()).index(x)]) for x in range(2,7)];


movi
film
one
like
time


### Save `word_dict`

Later on when we construct an endpoint which processes a submitted review we will need to make use of the `word_dict` which we have created. As such, we will save it to a file now for future use.

In [19]:
data_dir = 'dict_data/pytorch' # The folder we will use for storing data
if not os.path.exists(data_dir): # Make sure that the folder exists
    os.makedirs(data_dir)

In [20]:
with open(os.path.join(data_dir, 'word_dict.pkl'), "wb") as f:
    pickle.dump(word_dict, f)

### Transform the reviews

Now that we have our word dictionary which allows us to transform the words appearing in the reviews into integers, it is time to make use of it and convert our reviews to their integer sequence representation, making sure to pad or truncate to a fixed length, which in our case is `500`.

In [21]:
test_X

[['would',
  'given',
  'movi',
  '1',
  'laugh',
  'hard',
  'mani',
  'time',
  'give',
  'littl',
  'credit',
  'chanc',
  'film',
  'suppos',
  'funni',
  'movi',
  'bad',
  'think',
  'chimp',
  'wrote',
  'wish',
  'chimp',
  'written',
  'dialogu',
  'can',
  'cockroach',
  'surviv',
  'come',
  'nuclear',
  'holocaust',
  'movi',
  'exagger',
  'aw',
  'interspl',
  'scene',
  'origin',
  'carri',
  'realli',
  'good',
  'film',
  'scene',
  'one',
  'like',
  'interspl',
  'scene',
  'taxi',
  'driver',
  'babi',
  'genius',
  'rent',
  'alon',
  'enjoy',
  'experi',
  'need',
  'someon',
  'next',
  'confirm',
  'bad',
  'view',
  'worst',
  'actress',
  'millenium',
  'goe',
  'poor',
  'poor',
  'ami',
  'irv',
  'stone',
  'face',
  'monoton',
  'disastr',
  'wooden',
  'school',
  'counselor',
  'worst',
  'movi',
  'year',
  'far',
  'see',
  'also',
  'arlington',
  'road',
  'frink',
  '3'],
 ['firm',
  'believ',
  'film',
  'tv',
  'serial',
  'form',
  'art',
  'woul

In [72]:
def convert_and_pad(word_dict, sentence, pad=500):
    NOWORD = 0 # We will use 0 to represent the 'no word' category
    INFREQ = 1 # and we use 1 to represent the infrequent words, i.e., words not appearing in word_dict
    
    working_sentence = [NOWORD] * pad
    
    for word_index, word in enumerate(sentence[:pad]):
        if word in word_dict:
            working_sentence[word_index] = word_dict[word]
        else:
            working_sentence[word_index] = INFREQ
            
    return working_sentence, min(len(sentence), pad)

def convert_and_pad_data(word_dict, data, pad=500):
    """Applies the convert_and_pad function the entirety of the data"""
    result = []
    lengths = []
    if type(data[0])==list:
        for sentence in data:
            converted, leng = convert_and_pad(word_dict, sentence, pad)
            result.append(converted)
            lengths.append(leng)
            
    else:
        converted, leng = convert_and_pad(word_dict, data, pad)
        result.append(converted)
        lengths.append(leng)
        
        
    return np.array(result), np.array(lengths)

In [23]:
train_X_arr, train_X_len = convert_and_pad_data(word_dict, train_X)
test_X_arr, test_X_len = convert_and_pad_data(word_dict, test_X)

As a quick check to make sure that things are working as intended, check to see what one of the reviews in the training set looks like after having been processeed. Does this look reasonable? What is the length of a review in the training set?

In [24]:
# Use this cell to examine one of the processed reviews to make sure everything is working as intended.
train_X[23]

['third',
 'remak',
 'sleep',
 'eniemi',
 'yaarana',
 '1995',
 'agnisakshi',
 '1996',
 'agnisakshi',
 'one',
 'work',
 'better',
 'filmdaraar',
 'direct',
 'abba',
 'mustan',
 'sadli',
 'fail',
 'attempt',
 'hereth',
 'stori',
 'good',
 'handl',
 'good',
 'heroin',
 'shown',
 'regress',
 'climax',
 'disappointingdirect',
 'bad',
 'music',
 'goodrishi',
 'repris',
 'role',
 'yaarana',
 'strang',
 'also',
 'remak',
 'swte',
 'look',
 'fat',
 'lead',
 'okay',
 'juhi',
 'decent',
 'arbaaz',
 'tri',
 'hard',
 'debut',
 'manag',
 'mani',
 'scene',
 'chill',
 'audienc',
 'voic',
 'terribl',
 'johni',
 'loud']

In [25]:
train_X_arr[:, 24]

array([3706,  387,   27, ..., 1118,    1, 1030])

The vocabulary dictionary was created purely from the training data set. When we call the two functions above on our test data, if there is new, significantly useful vocabulary not present in the training set, this will simply be put down as a 1 - infrequent word. Assuming that the train data set's vocabulary was general enough to capture the most important words, this will hopefully not be a problem. However, we might need to update our vocabulary dictionary periodically to keep up with any real-time changes in data. 

## Step 3: Upload the data to S3

We're saving our data locally and upload it to AWS' Simple Storage Service (S3). 

### Save the processed training dataset locally

It is important to note the format of the data that we are saving as we will need to know it when we write the training code. In our case, each row of the dataset has the form `label`, `length`, `review[500]` where `review[500]` is a sequence of `500` integers representing the words in the review.

In [26]:
import pandas as pd
    
pd.concat([pd.DataFrame(train_y), pd.DataFrame(train_X_len), pd.DataFrame(train_X_arr)], axis=1) \
        .to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)

In [27]:
pd.concat([pd.DataFrame(test_y), pd.DataFrame(test_X_len), pd.DataFrame(test_X_arr)], axis=1) \
        .to_csv(os.path.join(data_dir, 'test.csv'), header=False, index=False)

In [28]:
# removing the train data because the imdb dataset is causing my nb instance to run slowly. I delete the test data after Step 6
!cd; cd SageMaker/sagemaker-deployment/data/aclImdb; rm -r train


### Uploading the training data


Next, we need to upload the training data to the SageMaker default S3 bucket so that we can provide access to it while training our model.

It's in the following cell of code that we start to make greater use of Sagemaker. 

In [44]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/sentiment_rnn'

role = sagemaker.get_execution_role()

In [45]:
input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)

In [46]:
vocab_dict = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)

In [29]:
import boto3
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

data_location = 's3://sagemaker-eu-west-2-363162872357/sagemaker/sentiment_rnn/train.csv'

train_data = pd.read_csv(data_location, header=None, names=None, nrows=250)

In [30]:
train_data.head()

,0,1,2,3,4,5,6,7,8,9,...,492,493,494,495,496,497,498,499,500,501
0,0,138,3,276,776,1000,5,37,200,389,...,0,0,0,0,0,0,0,0,0,0
1,0,94,16,238,148,130,761,287,2,281,...,0,0,0,0,0,0,0,0,0,0
2,1,58,732,4,55,20,1444,2083,53,683,...,0,0,0,0,0,0,0,0,0,0
3,0,103,2,20,2720,2967,3028,577,4,79,...,0,0,0,0,0,0,0,0,0,0
4,1,94,45,575,2,334,541,420,1023,361,...,0,0,0,0,0,0,0,0,0,0


**NOTE:** The cell above uploads the entire contents of our data directory. This includes the `word_dict.pkl` file. This is fortunate as we will need this later on when we create an endpoint that accepts an arbitrary review. For now, we will just take note of the fact that it resides in the data directory (and so also in the S3 training bucket) and that we will need to make sure it gets saved in the model directory.

## Step 4: Build and Train the PyTorch Model

A model in Sagemaker comprises three objects

 - Model Artifacts,
 - Training Code, and
 - Inference Code,
 
each of which interact with one another. 

We will start by implementing our own neural network in PyTorch along with a training script. For the purposes of this project we have provided the necessary model object in the `model.py` file, inside of the `train` folder. You can see the provided implementation by running the cell below.

In [216]:
!pygmentize train/model.py

import torch.nn as nn

class LSTMClassifier(nn.Module):
    """
    This is the simple RNN model we will be using to perform Sentiment Analysis.
    """

    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        """
        Initialize the model by settingg up the various layers.
        """
        super(LSTMClassifier, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.dense = nn.Linear(in_features=hidden_dim, out_features=1)
        self.sig = nn.Sigmoid()
        
        self.word_dict = None

    def forward(self, x):
        """
        Perform a forward pass of our model on some input.
        """
        x = x.t()
        lengths = x[0,:]
        reviews = x[1:,:]
        embeds = self.embedding(reviews)
        lstm_out, _ = self.lstm(embeds)
        out = self.dense(lstm_out)
        out = out[lengths - 1, range(len(lengths))]
        return self.sig

The important takeaway from the implementation provided is that there are three parameters that we may wish to tweak to improve the performance of our model. These are the embedding dimension, the hidden dimension and the size of the vocabulary. We will likely want to make these parameters configurable in the training script so that if we wish to modify them we do not need to modify the script itself. We will see how to do this later on. To start we will write some of the training code in the notebook so that we can more easily diagnose any issues that arise.

First we will load a small portion of the training data set to use as a sample. It would be very time consuming to try and train the model completely in the notebook as we do not have access to a gpu and the compute instance that we are using is not particularly powerful. However, we can work on a small bit of the data to get a feel for how our training script is behaving.

In [31]:
data_dir

'dict_data/pytorch'

In [33]:
import torch
import torch.utils.data

In [32]:
# Read in only the first 250 rows
# train_sample = pd.read_csv(os.path.join(data_dir, 'train.csv'), header=None, names=None, nrows=250)
train_sample = train_data[:250]
train_sample

,0,1,2,3,4,5,6,7,8,9,...,492,493,494,495,496,497,498,499,500,501
0,0,138,3,276,776,1000,5,37,200,389,...,0,0,0,0,0,0,0,0,0,0
1,0,94,16,238,148,130,761,287,2,281,...,0,0,0,0,0,0,0,0,0,0
2,1,58,732,4,55,20,1444,2083,53,683,...,0,0,0,0,0,0,0,0,0,0
3,0,103,2,20,2720,2967,3028,577,4,79,...,0,0,0,0,0,0,0,0,0,0
4,1,94,45,575,2,334,541,420,1023,361,...,0,0,0,0,0,0,0,0,0,0
5,0,32,2,700,105,397,376,841,397,15,...,0,0,0,0,0,0,0,0,0,0
6,1,91,272,505,3401,1296,1,541,902,544,...,0,0,0,0,0,0,0,0,0,0
7,0,135,240,716,294,1288,398,994,2907,3416,...,0,0,0,0,0,0,0,0,0,0
8,1,83,201,75,667,3907,80,690,2,457,...,0,0,0,0,0,0,0,0,0,0
9,1,131,3708,17,371,1576,1663,93,269,11,...,0,0,0,0,0,0,0,0,0,0


In [34]:
(train_sample.drop([0], axis=1).values)

array([[138,   3, 276, ...,   0,   0,   0],
       [ 94,  16, 238, ...,   0,   0,   0],
       [ 58, 732,   4, ...,   0,   0,   0],
       ...,
       [179,   1, 865, ...,   0,   0,   0],
       [106, 254,  12, ...,   0,   0,   0],
       [ 67,  16,  49, ...,   0,   0,   0]])

In [127]:
train_sample

,0,1,2,3,4,5,6,7,8,9,...,492,493,494,495,496,497,498,499,500,501
0,0,138,3,276,776,1000,5,37,200,389,...,0,0,0,0,0,0,0,0,0,0
1,0,94,16,238,148,130,761,287,2,281,...,0,0,0,0,0,0,0,0,0,0
2,1,58,732,4,55,20,1444,2083,53,683,...,0,0,0,0,0,0,0,0,0,0
3,0,103,2,20,2720,2967,3028,577,4,79,...,0,0,0,0,0,0,0,0,0,0
4,1,94,45,575,2,334,541,420,1023,361,...,0,0,0,0,0,0,0,0,0,0
5,0,32,2,700,105,397,376,841,397,15,...,0,0,0,0,0,0,0,0,0,0
6,1,91,272,505,3401,1296,1,541,902,544,...,0,0,0,0,0,0,0,0,0,0
7,0,135,240,716,294,1288,398,994,2907,3416,...,0,0,0,0,0,0,0,0,0,0
8,1,83,201,75,667,3907,80,690,2,457,...,0,0,0,0,0,0,0,0,0,0
9,1,131,3708,17,371,1576,1663,93,269,11,...,0,0,0,0,0,0,0,0,0,0


In [129]:
train_sample_X.shape

torch.Size([250, 501])

In [35]:
# Turn the input pandas dataframe into tensors
train_sample_y = torch.from_numpy(train_sample[[0]].values).float().squeeze()
train_sample_X = torch.from_numpy(train_sample.drop([0], axis=1).values).long()

# Build the dataset
train_sample_ds = torch.utils.data.TensorDataset(train_sample_X, train_sample_y)
# Build the dataloader
train_sample_dl = torch.utils.data.DataLoader(train_sample_ds, batch_size=50)

In [36]:
# Turthe input pandas dataframe into tensors
train_y = torch.from_numpy(train_data[[0]].values).float().squeeze()
train_X = torch.from_numpy(train_data.drop([0], axis=1).values).long()

# BuilXdataset
train_ds = torch.utils.data.TensorDataset(train_X, train_y)
# BuilXdataloader
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=50)

### Writing the training method

In [37]:
def train(model, train_loader, epochs, optimizer, loss_fn, device):
    for epoch in range(1, epochs + 1):
        #  sets the model in training mode
        model.train()
        # initialising the loss
        total_loss = 0
        # calling data loader for a new batch each loop
        for batch in train_loader:
            # dataloader gives us features and targets; assigning them
            batch_X, batch_y = batch
            
            # moving our tensors to the device
            # in pytorch this is useful for the purposes of controlling which 
            # hardware is doing the work
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
                        
            #clear any gradients
            optimizer.zero_grad()
            
            # generate model predictions
            y_hat = model(batch_X)
            
            # compute loss for this batch
            loss = loss_fn(y_hat, batch_y)
            
            # computes the derivative of loss with respect to 
            # each trainable parameter
            loss.backward()
            
            # update model weights
            optimizer.step()
            
            total_loss += loss.data.item()
        print("Epoch: {}, BCELoss: {}".format(epoch, total_loss / len(train_loader)))

#### REFERENCE:

Due to my knowledge of pytorch being rusty, I read through the blog linked below from Jason Brownlee's Machine Learning Mastery to reacquaint myself with how pytorch functions.
https://machinelearningmastery.com/pytorch-tutorial-develop-deep-learning-models/ 

Supposing we have the training method above, we will test that it is working by writing a bit of code in the notebook that executes our training method on the small sample training set that we loaded earlier. The reason for doing this in the notebook is so that we have an opportunity to fix any errors that arise early when they are easier to diagnose.

In [38]:
import torch.optim as optim
from train.model import LSTMClassifier

In [39]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMClassifier(32, 100, 5000).to(device)
optimizer = optim.Adam(model.parameters())
loss_fn = torch.nn.BCELoss()

train(model, train_sample_dl, 5, optimizer, loss_fn, device)

In order to construct a PyTorch model using SageMaker we must provide SageMaker with a training script. We may optionally include a directory which will be copied to the container and from which our training code will be run. When the training container is executed it will check the uploaded directory (if there is one) for a `requirements.txt` file and install any required Python libraries, after which the training script will be run.

### Training the model

When a PyTorch model is constructed in SageMaker, an entry point must be specified. This is the Python file which will be executed when the model is trained. Inside of the `train` directory is a file called `train.py` which has been provided and which contains most of the necessary code to train our model. The only thing that is missing is the implementation of the `train()` method which we wrote earlier in this notebook.

The way that SageMaker passes hyperparameters to the training script is by way of arguments. These arguments can then be parsed and used in the training script. To see how this is done take a look at the provided `train/train.py` file.

In [40]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point="train.py",
                    source_dir="train",
                    role=role,
                    framework_version='0.4.0',
                    train_instance_count=1,
                    train_instance_type='ml.m4.xlarge',
                    hyperparameters={
                        'epochs': 10,
                        'hidden_dim': 200,
                    })

In [47]:
input_data

's3://sagemaker-eu-west-2-363162872357/sagemaker/sentiment_rnn'

In [ ]:
estimator.fit({'training': input_data})

2020-06-03 09:38:36 Starting - Starting the training job...
2020-06-03 09:38:39 Starting - Launching requested ML instances.........
2020-06-03 09:40:11 Starting - Preparing the instances for training......
2020-06-03 09:41:33 Downloading - Downloading input data
2020-06-03 09:41:33 Training - Downloading the training image...
2020-06-03 09:41:53 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-06-03 09:41:53,869 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-06-03 09:41:53,872 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-06-03 09:41:53,885 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-06-03 09:41:56,923 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-06-03 09:41:57,162 sagemaker-containers INF

## Step 5: Testing the model

As mentioned at the top of this notebook, we will be testing this model by first deploying it and then sending the testing data to the deployed endpoint. We will do this so that we can make sure that the deployed model is working correctly.

## Step 6: Deploy the model for testing

Now that we have trained our model, we would like to test it to see how it performs. Currently our model takes input of the form `review_length, review[500]` where `review[500]` is a sequence of `500` integers which describe the words present in the review, encoded using `word_dict`. Fortunately for us, SageMaker provides built-in inference code for models with simple inputs such as this.

There is one thing that we need to provide, however, and that is a function which loads the saved model. This function must be called `model_fn()` and takes as its only parameter a path to the directory where the model artifacts are stored. This function must also be present in the python file which we specified as the entry point. In our case the model loading function has been provided and so no changes need to be made.

**NOTE**: When the built-in inference code is run it must import the `model_fn()` method from the `train.py` file. This is why the training code is wrapped in a main guard ( ie, `if __name__ == '__main__':` )

Since we don't need to change anything in the code that was uploaded during training, we can simply deploy the current model as-is.
 When deploying a model you are asking SageMaker to launch an compute instance that will wait for data to be sent to it. As a result, this compute instance will continue to run until *you* shut it down. This is important to know since the cost of a deployed endpoint depends on how long it has been running for.

In other words **If you are no longer using a deployed endpoint, shut it down!**



In [49]:
# once this is done sagemaker will have created a VM that contains our trained model at a specific URL,
# accessible to us and to our app
predictor = estimator.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')


-------------!

## Step 7 - Use the model for testing

Once deployed, we can read in the test data and send it off to our deployed model to get some results. Once we collect all of the results we can determine how accurate our model is.

In [50]:
test_X = pd.concat([pd.DataFrame(test_X_len), pd.DataFrame(test_X_arr)], axis=1)

In [51]:
train_X_arr

array([[   3,  276,  776, ...,    0,    0,    0],
       [  16,  238,  148, ...,    0,    0,    0],
       [ 732,    4,   55, ...,    0,    0,    0],
       ...,
       [4980, 2291,    1, ...,    0,    0,    0],
       [ 499,  900,  159, ...,    0,    0,    0],
       [  47,  619,  786, ...,    0,    0,    0]])

In [52]:
test_X.values

array([[  84,   15,  303, ...,    0,    0,    0],
       [ 135, 4007,   94, ...,    0,    0,    0],
       [ 337,   69,   12, ...,    0,    0,    0],
       ...,
       [  52,  186,  537, ...,    0,    0,    0],
       [  44,    3,   47, ...,    0,    0,    0],
       [ 128, 3068,   52, ...,    0,    0,    0]])

In [223]:
test_X_arr[0]

array([  15,  303,    2,  229,  155,  178,   46,    6,   57,   52,  428,
        456,    3,  278,   84,    2,   24,   30,    1,  950,  426,    1,
        343,  338,    1,    1,  883,   45, 3004, 4749,    2, 2233,  299,
          1,   18,   90,  592,   16,    7,    3,   18,    4,    5,    1,
         18, 3832, 1995,  731,    1,  443,  510,   77,  365,  122,  201,
        323, 3293,   24,  234,  174,  352,    1,  194,  288,  288, 3175,
          1, 1158,  213, 4095, 4476, 1495,  318,    1,  174,    2,   44,
        151,   11,   27,    1, 1123,    1,  289,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [55]:
# We split the data into chunks and send each chunk seperately, accumulating the results.

def predict(data, rows=512):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = np.array([])
    for array in split_array:
        predictions = np.append(predictions, predictor.predict(array))
    
    return predictions

In [ ]:
predictions = predict(test_X.values)
predictions = [round(num) for num in predictions]

In [57]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, predictions)

0.85412

### More testing

We now have a trained model which has been deployed and which we can send processed reviews to and which returns the predicted sentiment. However, ultimately we would like to be able to send our model an unprocessed review. That is, we would like to send the review itself as a string. For example, suppose we wish to send the following review to our model.

In [58]:
test_review = 'The simplest pleasures in life are the best, and this film is one of them. Combining a rather basic storyline of love and adventure this movie transcends the usual weekend fair with wit and unmitigated charm.'

The question we now need to answer is, how do we send this review to our model?

Recall in the first section of this notebook we did a bunch of data processing to the IMDb dataset. In particular, we did two specific things to the provided reviews.
 - Removed any html tags and stemmed the input
 - Encoded the review as a sequence of integers using `word_dict`
 
In order to process the review we will need to repeat these two steps.

In [59]:
# TODO: Convert test_review into a form usable by the model and save the results in test_data
test_data_prep = review_to_words(test_review)
len(test_data_prep)

20

In [60]:
test_data_prep

['simplest',
 'pleasur',
 'life',
 'best',
 'film',
 'one',
 'combin',
 'rather',
 'basic',
 'storylin',
 'love',
 'adventur',
 'movi',
 'transcend',
 'usual',
 'weekend',
 'fair',
 'wit',
 'unmitig',
 'charm']

In [132]:
test_data = (convert_and_pad_data(word_dict, test_data_prep))
len(test_data[0])

1

Now that we have processed the review, we can send the resulting array to our model to predict the sentiment of the review.

In [134]:
test_data[1].shape

(1,)

In [154]:
# test_data_arr = torch.from_numpy(test_data[0]).reshape(-1,)
# test_data_len = torch.from_numpy(test_data[1]).reshape(-1,)

In [155]:
# test_data_arr.shape

torch.Size([500])

In [226]:
predictor.predict((test_data[0]))

b'1.0'

### Delete the endpoint

Once we've deployed an endpoint it continues to run until we tell it to shut down. Since we are done using our endpoint for now, we can delete it.

In [79]:
estimator.delete_endpoint()

## Step 6 (again) - Deploy the model for the web app

Now that we know that our model is working, it's time to create some custom inference code so that we can send the model a review which has not been processed and have it determine the sentiment of the review.

As we saw above, by default the estimator which we created, when deployed, will use the entry script and directory which we provided when creating the model. However, since we now wish to accept a string as input and our model expects a processed review, we need to write some custom inference code.

We will store the code that we write in the `serve` directory. Provided in this directory is the `model.py` file that we used to construct our model, a `utils.py` file which contains the `review_to_words` and `convert_and_pad` pre-processing functions which we used during the initial data processing, and `predict.py`, the file which will contain our custom inference code. Note also that `requirements.txt` is present which will tell SageMaker what Python libraries are required by our custom inference code.

When deploying a PyTorch model in SageMaker, you are expected to provide four functions which the SageMaker inference container will use.
 - `model_fn`: This function is the same function that we used in the training script and it tells SageMaker how to load our model.
 - `input_fn`: This function receives the raw serialized input that has been sent to the model's endpoint and its job is to de-serialize and make the input available for the inference code.
 - `output_fn`: This function takes the output of the inference code and its job is to serialize this output and return it to the caller of the model's endpoint.
 - `predict_fn`: The heart of the inference script, this is where the actual prediction is done and is the function which you will need to complete.

For the simple website that we are constructing during this project, the `input_fn` and `output_fn` methods are relatively straightforward. We only require being able to accept a string as input and we expect to return a single value as output. You might imagine though that in a more complex application the input or output may be image data or some other binary data which would require some effort to serializ

### Writing inference code

Before writing our custom inference code, we will begin by taking a look at the code which has been provided.

In [80]:
!pygmentize serve/predict.py

import argparse
import json
import os
import pickle
import sys
import sagemaker_containers
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data

from model import LSTMClassifier

from utils import review_to_words, convert_and_pad

def model_fn(model_dir):
    """Load the PyTorch model from the `model_dir` directory."""
    print("Loading model.")

    # First, load the parameters used to create the model.
    model_info = {}
    model_info_path = os.path.join(model_dir, 'model_info.pth')
    with open(model_info_path, 'rb') as f:
        model_info = torch.load(f)

    print("model_info: {}".format(model_info))

    # Determine the device and construct the model.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = LSTMClassifier(model_info['embedding_dim'], model_info['hidden_dim'], model_info['vocab_size'])

    # Load the store model parameters.
    model_path = os.path.join(mod

As mentioned earlier, the `model_fn` method is the same as the one provided in the training code and the `input_fn` and `output_fn` methods are very simple and your task will be to complete the `predict_fn` method. Make sure that you save the completed file as `predict.py` in the `serve` directory.


### Deploying the model

Now that the custom inference code has been written, we will create and deploy our model. To begin with, we need to construct a new PyTorchModel object which points to the model artifacts created during training and also points to the inference code that we wish to use. Then we can call the deploy method to launch the deployment container.

**NOTE**: The default behaviour for a deployed PyTorch model is to assume that any input passed to the predictor is a `numpy` array. In our case we want to send a string so we need to construct a simple wrapper around the `RealTimePredictor` class to accomodate simple strings. In a more complicated situation you may want to provide a serialization object, for example if you wanted to sent image data.

In [204]:
from sagemaker.predictor import RealTimePredictor
from sagemaker.pytorch import PyTorchModel

class StringPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(StringPredictor, self).__init__(endpoint_name, sagemaker_session, content_type='text/plain')

model = PyTorchModel(model_data=estimator.model_data, # pointing to the model artefacts - our learned weights and coeffs
                     role = role, # using the specified ARN
                     framework_version='0.4.0', 
                     entry_point='predict.py', # which file to go to to find the respective functions
                     source_dir='serve', # directory to access
                     predictor_cls=StringPredictor) 
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

-------------!

### Testing the model

Now that we have deployed our model with the custom inference code, we should test to see if everything is working. Here we test our model by loading the first `250` positive and negative reviews and send them to the endpoint, then collect the results. The reason for only sending some of the data is that the amount of time it takes for our model to process the input and then perform inference is quite long and so testing the entire data set would be prohibitive.

In [205]:
import glob

def test_reviews(data_dir='../data/aclImdb', stop=10):
    
    results = []
    ground = []
    
    # We make sure to test both positive and negative reviews    
    for sentiment in ['pos', 'neg']:
        
        path = os.path.join(data_dir, 'test', sentiment, '*.txt')
        files = glob.glob(path)
        
        files_read = 0
        
        print('Starting ', sentiment, ' files')
        
        # Iterate through the files and send them to the predictor
        for f in files:
            with open(f) as review:
                # First, we store the ground truth (was the review positive or negative)
                if sentiment == 'pos':
                    ground.append(1)
                else:
                    ground.append(0)
                # Read in the review and convert to 'utf-8' for transmission via HTTP
                review_input = review.read().encode('utf-8')
                # Send the review to the predictor and store the results
                results.append(int(float(predictor.predict(review_input))))
                
            # Sending reviews to our endpoint one at a time takes a while so we
            # only send a small number of reviews
            files_read += 1
            if files_read == stop:
                break
            
    return ground, results

In [206]:
ground, results = test_reviews()

Starting  pos  files
Starting  neg  files


In [212]:
from sklearn.metrics import accuracy_score
accuracy_score(ground, results)

0.45

As an additional test, we can try sending the `test_review` that we looked at earlier.

In [213]:
predictor.predict(test_review)

b'1.0'

In [215]:
# !cd; cd SageMaker/sagemaker-deployment/data; rm -r aclImdb

Now that we know our endpoint is working as expected, we can set up the web page that will interact with it. If you don't have time to finish the project now, make sure to skip down to the end of this notebook and shut down your endpoint. You can deploy it again when you come back.

## Step 7 (again): Use the model for the web app

So far we have been accessing our model endpoint by constructing a predictor object which uses the endpoint and then just using the predictor object to perform inference. What if we wanted to create a web app which accessed our model? The way things are set up currently makes that not possible since in order to access a SageMaker endpoint the app would first have to authenticate with AWS using an IAM role which included access to SageMaker endpoints. However, there is an easier way! We just need to use some additional AWS services.

The flowchart in the README should provide some idea of how the various AWS components interact with our model. 

A webapp is hosted (in our case our local is acting as a server). A user submits a review to it and the app accesses the URL of our API (using the POST method). The API then accesses the Lambda function, which we've coded to interact with the Sagemaker endpoint. Once data is sent to our endpoint, it is preprocessed and fed into the model's ```predict_fn```, which outputs a result (0 or 1). This is then sent back via the Lambda function, our API and ultimately to our webapp, which turns a 1 into "Your review was POSITIVE" and a 0 into "Your review was NEGATIVE". All of this was set up via AWS' Console. 


Udacity-provided instructions and code for the lambda function and the API Gateway set up.
### Setting up a Lambda function

The first thing we are going to do is set up a Lambda function. This Lambda function will be executed whenever our public API has data sent to it. When it is executed it will receive the data, perform any sort of processing that is required, send the data (the review) to the SageMaker endpoint we've created and then return the result.

#### Part A: Create an IAM Role for the Lambda function

Since we want the Lambda function to call a SageMaker endpoint, we need to make sure that it has permission to do so. To do this, we will construct a role that we can later give the Lambda function.

Using the AWS Console, navigate to the **IAM** page and click on **Roles**. Then, click on **Create role**. Make sure that the **AWS service** is the type of trusted entity selected and choose **Lambda** as the service that will use this role, then click **Next: Permissions**.

In the search box type `sagemaker` and select the check box next to the **AmazonSageMakerFullAccess** policy. Then, click on **Next: Review**.

Lastly, give this role a name. Make sure you use a name that you will remember later on, for example `LambdaSageMakerRole`. Then, click on **Create role**.

#### Part B: Create a Lambda function

Now it is time to actually create the Lambda function.

Using the AWS Console, navigate to the AWS Lambda page and click on **Create a function**. When you get to the next page, make sure that **Author from scratch** is selected. Now, name your Lambda function, using a name that you will remember later on, for example `sentiment_analysis_func`. Make sure that the **Python 3.6** runtime is selected and then choose the role that you created in the previous part. Then, click on **Create Function**.

On the next page you will see some information about the Lambda function you've just created. If you scroll down you should see an editor in which you can write the code that will be executed when your Lambda function is triggered. In our example, we will use the code below. 

```python
# We need to use the low-level library to interact with SageMaker since the SageMaker API
# is not available natively through Lambda.
import boto3

def lambda_handler(event, context):

    # The SageMaker runtime is what allows us to invoke the endpoint that we've created.
    runtime = boto3.Session().client('sagemaker-runtime')

    # Now we use the SageMaker runtime to invoke our endpoint, sending the review we were given
    response = runtime.invoke_endpoint(EndpointName = '**ENDPOINT NAME HERE**',    # The name of the endpoint we created
                                       ContentType = 'text/plain',                 # The data format that is expected
                                       Body = event['body'])                       # The actual review

    # The response is an HTTP response whose body contains the result of our inference
    result = response['Body'].read().decode('utf-8')

    return {
        'statusCode' : 200,
        'headers' : { 'Content-Type' : 'text/plain', 'Access-Control-Allow-Origin' : '*' },
        'body' : result
    }
```

Once you have copy and pasted the code above into the Lambda code editor, replace the `**ENDPOINT NAME HERE**` portion with the name of the endpoint that we deployed earlier. You can determine the name of the endpoint using the code cell below.

In [214]:
predictor.endpoint

'sagemaker-pytorch-2020-06-03-18-15-54-217'

Once you have added the endpoint name to the Lambda function, click on **Save**. Your Lambda function is now up and running. Next we need to create a way for our web app to execute the Lambda function.

### Setting up API Gateway

Now that our Lambda function is set up, it is time to create a new API using API Gateway that will trigger the Lambda function we have just created.

Using AWS Console, navigate to **Amazon API Gateway** and then click on **Get started**.

On the next page, make sure that **New API** is selected and give the new api a name, for example, `sentiment_analysis_api`. Then, click on **Create API**.

Now we have created an API, however it doesn't currently do anything. What we want it to do is to trigger the Lambda function that we created earlier.

Select the **Actions** dropdown menu and click **Create Method**. A new blank method will be created, select its dropdown menu and select **POST**, then click on the check mark beside it.

For the integration point, make sure that **Lambda Function** is selected and click on the **Use Lambda Proxy integration**. This option makes sure that the data that is sent to the API is then sent directly to the Lambda function with no processing. It also means that the return value must be a proper response object as it will also not be processed by API Gateway.

Type the name of the Lambda function you created earlier into the **Lambda Function** text entry box and then click on **Save**. Click on **OK** in the pop-up box that then appears, giving permission to API Gateway to invoke the Lambda function you created.

The last step in creating the API Gateway is to select the **Actions** dropdown and click on **Deploy API**. You will need to create a new Deployment stage and name it anything you like, for example `prod`.

You have now successfully set up a public API to access your SageMaker model. Make sure to copy or write down the URL provided to invoke your newly created public API as this will be needed in the next step. This URL can be found at the top of the page, highlighted in blue next to the text **Invoke URL**.

## Step 4: Deploying our web app

Now that we have a publicly available API, we can start using it in a web app. For our purposes, we have provided a simple static html file which can make use of the public api you created earlier.

In the `website` folder there should be a file called `index.html`. Download the file to your computer and open that file up in a text editor of your choice. There should be a line which contains **\*\*REPLACE WITH PUBLIC API URL\*\***. Replace this string with the url that you wrote down in the last step and then save the file.

Now, if you open `index.html` on your local computer, your browser will behave as a local web server and you can use the provided site to interact with your SageMaker model.

If you'd like to go further, you can host this html file anywhere you'd like, for example using github or hosting a static site on Amazon's S3. Once you have done this you can share the link with anyone you'd like and have them play with it too!

> **Important Note** In order for the web app to communicate with the SageMaker endpoint, the endpoint has to actually be deployed and running. This means that you are paying for it. Make sure that the endpoint is running when you want to use the web app but that you shut it down when you don't need it, otherwise you will end up with a surprisingly large AWS bill.


Couple of examples:

1. "Goodness gracious, did I really waste my life watching this incredibly sickening trite?" - Prediction: Negative
2. "awfully good; terribly fantastic; an appalingly good time was had" - Prediction: Positive


### Delete the endpoint

Remember to always shut down your endpoint if you are no longer using it. You are charged for the length of time that the endpoint is running so if you forget and leave it on you could end up with an unexpectedly large bill.

In [227]:
predictor.delete_endpoint()